<a href="https://colab.research.google.com/github/gustavo-ifusp/prefeitura_smit_sp360/blob/main/Passageiros_transportados_an%C3%A1lise_por_dia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [ ]:
import bs4
from requests import get
from bs4 import BeautifulSoup
import time
import re
import gspread
print("BeautifulSoup ->", bs4.__version__)

BeautifulSoup -> 4.6.3


#**Passareiros Transportados SPTRANS (concluído)**
**Link dos Dados de origem:**
https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932<br>
<br>**Notebook da leitura retroativa:**<br>
https://colab.research.google.com/drive/1R_gGCuGN-pZbpFbyH0Bv5A_ZNv3yenur?usp=sharing
<br>**Notebook de automação da leitura:**<br>
https://colab.research.google.com/drive/1KbqjyNzNn4J9JvsQO-HthIg7cnJI3ePJ?usp=sharing
<br>**Arquivos:**<br>
https://drive.google.com/drive/folders/16F_gT2tB9lF0ToIh1ftqKnbtrwHrty4X?usp=sharing

<br>**Planilhas prontas:**<br>
https://drive.google.com/file/d/1GS5DIIp2o1nL3o4guXATG0cC0vUdNeLg/view?usp=sharing

<br>**Por ano:**<br>
https://drive.google.com/file/d/1rh9w24382SKv_2qH2F0-qJxBs6ZdKfSV/view?usp=sharing
https://drive.google.com/file/d/1azjjQsAVRtQqqdiYAGWNxNMWnAesulen/view?usp=sharing
https://drive.google.com/file/d/1eAqgEP9gDS6SP5ZO4_ybg8Gug69UsBnI/view?usp=sharing

<br>**Por Mês:**<br>
https://drive.google.com/drive/folders/115-tVXZYdL82MfoUpxKAIc7zlzAJfF9q?usp=sharing

#**Solução simplificada para qualquer dia, basta modificar o dia na 'url1':** <br>check - funcionou perfeitamente

In [ ]:
url1 = "https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/11AGO2021(1).xls"
dados = pd.read_excel(url1, skiprows = range(0, 2))
dados.sample(2)

,Data,Grupo,Lote,Empresa,Linha,Passageiros Pagtes Em Dinheiro,Passageiros Comum e VT,Passageiros Pgts Bu Comum M,Passageiros Pagtes Estudante,Passageiros Pgts Bu Est Mensal,Passageiros Pgts Bu Vt Mensal,Passageiros Pagantes,Passageiros Int Ônibus->Ônibus,Passageiros Com Gratuidade,Passageiros Com Gratuidade Est,Tot Passageiros Transportados
48,11/08/2021,GRUPO ARTICULAÇÃO,AR1,Santa Brigida Mangalot,847J10 - CITY JARAGUA/JAGUARE,342,2359,69,39,0,0,2809,898,273,59,4039
1092,11/08/2021,GRUPO ESTRUTURAL,E01,Santa Brigida,805021 - ANHANGUERA/LAPA,15,320,7,6,0,0,348,24,17,15,404


In [ ]:
dados

,Data,Tipo,Area,Empresa,Linha,Passageiros Pagtes Em Dinheiro,Passageiros Comum e VT,Passageiros Pgts Bu Comum M,Passageiros Pagtes Estudante,Passageiros Pgts Bu Est Mensal,Passageiros Pgts Bu Vt Mensal,Passageiros Pagantes,Passageiros Int Ônibus->Ônibus,Passageiros Com Gratuidade,Passageiros Com Gratuidade Est,Tot Passageiros Transportados
0,2021-05-27,CONCESSAO,AREA 2,201,N40211 - METRO ITAQUERA/TERM VL CARRAO,1,31,2,0,0,0,34,3,3,0,40
1,2021-05-27,CONCESSAO,AREA 2,201,N40511 - TERM V CARRAO/METRO ITAQUERA,9,38,2,0,0,0,49,17,3,0,69
2,2021-05-27,CONCESSAO,AREA 2,201,200210 - TERM P D PEDRO II/TERM BANDEIR,15,478,39,2,0,0,534,1012,251,17,1814
3,2021-05-27,CONCESSAO,AREA 2,201,210010 - TERM V CARRAO/PCA DA SE CIRC,560,3563,209,44,0,0,4376,2890,1170,73,8509
4,2021-05-27,CONCESSAO,AREA 2,201,210110 - PCA SILVIO ROMERO/PCA DA SE,20,98,0,0,0,0,118,34,28,1,181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,2021-05-27,PERMISSAO,AREA 1,124,802510 - JD ROSA MARIA/BUTANTA,95,1010,33,9,0,0,1147,623,275,22,2067
1246,2021-05-27,PERMISSAO,AREA 1,124,802610 - JD INGA/BUTANTA,99,1109,38,12,0,0,1258,409,127,21,1815
1247,2021-05-27,PERMISSAO,AREA 1,124,803810 - JAGUARE/LAPA,622,4796,118,16,0,0,5552,2032,771,89,8444
1248,2021-05-27,PERMISSAO,AREA 1,190,9940PR - TERMINAL GRAJAU - 6726-10,14,829,31,2,0,0,876,377,30,17,1300


In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Data                            1250 non-null   datetime64[ns]
 1   Tipo                            1250 non-null   object        
 2   Area                            1250 non-null   object        
 3   Empresa                         1250 non-null   int64         
 4   Linha                           1250 non-null   object        
 5   Passageiros Pagtes Em Dinheiro  1250 non-null   int64         
 6   Passageiros Comum e VT          1250 non-null   int64         
 7   Passageiros Pgts Bu Comum M     1250 non-null   int64         
 8   Passageiros Pagtes Estudante    1250 non-null   int64         
 9   Passageiros Pgts Bu Est Mensal  1250 non-null   int64         
 10  Passageiros Pgts Bu Vt Mensal   1250 non-null   int64         
 11  Pass

In [ ]:
a= dados['Passageiros Pagtes Em Dinheiro']
b = dados['Passageiros Comum e VT']
c = dados['Passageiros Pgts Bu Comum M']
d = dados['Passageiros Pagtes Estudante']
e = dados['Passageiros Pgts Bu Est Mensal']
f = dados['Passageiros Pgts Bu Vt Mensal']
g = dados['Passageiros Pagantes']
h = dados['Passageiros Int Ônibus->Ônibus']
i = dados['Passageiros Com Gratuidade']
j = dados['Passageiros Com Gratuidade Est']
t = dados['Tot Passageiros Transportados']
i1 = pd.concat([a,b,c,d,e,f,g,h,i,j,t],axis = 1)
i1 = i1.fillna(0)
i1.rename(columns={'Passageiros Pagtes Em Dinheiro':'a'}, inplace = True)
i1.rename(columns={'Passageiros Comum e VT':'b'}, inplace = True)
i1.rename(columns={'Passageiros Pgts Bu Comum M':'c'}, inplace = True)
i1.rename(columns={'Passageiros Pagtes Estudante':'d'}, inplace = True)
i1.rename(columns={'Passageiros Pgts Bu Est Mensal':'e'}, inplace = True)
i1.rename(columns={'Passageiros Pgts Bu Vt Mensal':'f'}, inplace = True)
i1.rename(columns={'Passageiros Pagantes':'g'}, inplace = True)
i1.rename(columns={'Passageiros Int Ônibus->Ônibus':'h'}, inplace = True)
i1.rename(columns={'Passageiros Com Gratuidade':'i'}, inplace = True)
i1.rename(columns={'Passageiros Com Gratuidade Est':'j'}, inplace = True)
i1.rename(columns={'Tot Passageiros Transportados':'t'}   , inplace = True) 
i1 = i1.infer_objects()           
i1.sample(2)

,a,b,c,d,e,f,g,h,i,j,t
1205,4,154,5,1,0,0,164,6,14,0,184
613,18,261,8,2,0,0,289,65,16,4,374


In [ ]:
i1.describe()

,a,b,c,d,e,f,g,h,i,j,t
count,1250.000000,1250.00000,1250.00000,1250.000000,1250.000000,1250.00000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000
mean,174.380000,1817.06640,75.55200,16.123200,0.041600,0.00480,2083.168000,1045.233600,376.028800,37.931200,3542.361600
std,204.792814,1866.53142,87.98076,18.380792,0.233051,0.08935,2137.879157,1335.973563,374.512246,42.362707,3739.041014
min,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24.000000,376.25000,13.25000,2.000000,0.000000,0.00000,422.500000,172.000000,70.000000,6.000000,712.000000
50%,105.000000,1309.00000,50.50000,11.000000,0.000000,0.00000,1514.500000,679.000000,298.500000,25.000000,2625.500000
75%,250.500000,2608.75000,105.75000,23.000000,0.000000,0.00000,3006.000000,1388.250000,552.000000,55.000000,4964.750000
max,1472.000000,13128.00000,891.00000,137.000000,2.000000,2.00000,14782.000000,16049.000000,2909.000000,357.000000,31082.000000


In [ ]:
soma_total = i1['t'].sum()
print("A soma total do dia é: ",soma_total, "passageiros.")

A soma total do dia é:  4427952 passageiros.


In [ ]:
a_total = i1['a'].sum()
b_total = i1['b'].sum()
c_total = i1['c'].sum()
d_total = i1['d'].sum()
e_total = i1['e'].sum()
f_total = i1['f'].sum()
g_total = i1['g'].sum()
h_total = i1['h'].sum()
i_total = i1['i'].sum()
j_total = i1['j'].sum()
print("A soma total do dia de Passageiros Pagtes Em Dinheiro é: ",a_total, "passagens")
print("A soma total do dia de Passageiros Comum e VT é: ",b_total, "passagens")
print("A soma total do dia de Passageiros Pgts Bu Comum M é: ",c_total, "passagens")
print("A soma total do dia de Passageiros Pagtes Estudante é: ",d_total, "passagens")
print("A soma total do dia de Passageiros Pgts Bu Est Mensal é: ",e_total, "passagens")
print("A soma total do dia de Passageiros Pgts Bu Vt Mensal é: ",f_total, "passagens")
print("A soma total do dia de Passageiros Pagantes é: ",g_total, "passagens")
print("A soma total do dia de Passageiros Int Ônibus->Ônibus é: ",h_total, "passagens")
print("A soma total do dia de Passageiros Com Gratuidade é: ",i_total, "passagens")
print("A soma total do dia de Passageiros Com Gratuidade é: ",j_total, "passagens")

A soma total do dia de Passageiros Pagtes Em Dinheiro é:  217975 passagens
A soma total do dia de Passageiros Comum e VT é:  2271333 passagens
A soma total do dia de Passageiros Pgts Bu Comum M é:  94440 passagens
A soma total do dia de Passageiros Pagtes Estudante é:  20154 passagens
A soma total do dia de Passageiros Pgts Bu Est Mensal é:  52 passagens
A soma total do dia de Passageiros Pgts Bu Vt Mensal é:  6 passagens
A soma total do dia de Passageiros Pagantes é:  2603960 passagens
A soma total do dia de Passageiros Int Ônibus->Ônibus é:  1306542 passagens
A soma total do dia de Passageiros Com Gratuidade é:  470036 passagens
A soma total do dia de Passageiros Com Gratuidade é:  47414 passagens


In [ ]:
total = a_total+b_total+c_total+d_total+e_total+f_total+g_total+h_total+i_total+j_total
print("A soma:", total,"tem que ser maior que", soma_total, "por um passageiro pertencer a mais de uma categoria (coluna)." )

A soma: 7031912 tem que ser maior que 4427952 por um passageiro pertencer a mais de uma categoria (coluna).


#**Começando com 2020 -  janeiro**

**Juntar os 2882 arquivos 'xls'. Utilizei o ParseHub para a web scraping, adcionando posteriormente o arquivos no Google Drive para abrir com facilidade aqui.** <br>Para não sobrecarregar os códigos e o Google Colab, usarei o poríodo de 1 mês para verificar a funcionalidade e depois carregarei no computador com todo conteúdo.

#**1hcW2zhLnhVZGICjxfbIs9vDaUtwUhp63** - ID Pasta Google driver
[https://drive.google.com/drive/folders/16F_gT2tB9lF0ToIh1ftqKnbtrwHrty4X?usp=sharing](https://)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
arquivos = glob.glob('/content/drive/My Drive/Prefeitura/Resultados/*.xls')
df = pd.DataFrame(arquivos)
arquivos

['/content/drive/My Drive/Prefeitura/Resultados/3-tabela_total_de_passageiros_transportados_2021.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/1-tabela_total_de_passageiros_transportados_2019.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/2-tabela_total_de_passageiros_transportados_2020.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/0-tabela_total_de_passageiros_transportados_2019_2020_2021.xls']

lendo o resutado!

In [ ]:
url='/content/drive/My Drive/Prefeitura/Resultados/0-tabela_total_de_passageiros_transportados_2019_2020_2021.xls'

In [ ]:
df1 = pd.read_excel(url)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Data                            879 non-null    datetime64[ns]
 1   Tot Passageiros Transportados   879 non-null    int64         
 2   Passageiros Pagtes Em Dinheiro  879 non-null    int64         
 3   Passageiros Comum e VT          879 non-null    int64         
 4   Passageiros Pgts Bu Comum M     879 non-null    int64         
 5   Passageiros Pagtes Estudante    879 non-null    int64         
 6   Passageiros Pgts Bu Est Mensal  879 non-null    int64         
 7   Passageiros Pgts Bu Vt Mensal   879 non-null    int64         
 8   Passageiros Pagantes            879 non-null    int64         
 9   Passageiros Int Ônibus->Ônibus  879 non-null    int64         
 10  Passageiros Com Gratuidade      879 non-null    int64         
 11  Passag

In [ ]:
x = len(arquivos)
print("Existem", x, "arquivos nessa lista.")

Existem 31 arquivos nessa lista.


In [ ]:
lin = df.iloc[30,0]
print("URL é:", lin)

URL é: /content/drive/My Drive/Prefeitura/compilado_base/2021/Janeiro/31JAN2021.xls


#**Teste 1 (check) - Funcionou perfeitamente!**

In [ ]:
all_data = pd.DataFrame() 
for f in arquivos:
    df1 = pd.read_excel(f,skiprows = range(0, 2))
    all_data = all_data.append(df1)
all_data.sample(2)

,Data,Tipo,Area,Empresa,Linha,Passageiros Pagtes Em Dinheiro,Passageiros Comum e VT,Passageiros Pgts Bu Comum M,Passageiros Pagtes Estudante,Passageiros Pgts Bu Est Mensal,Passageiros Pgts Bu Vt Mensal,Passageiros Pagantes,Passageiros Int Ônibus->Ônibus,Passageiros Com Gratuidade,Passageiros Com Gratuidade Est,Tot Passageiros Transportados
909,2020-01-01,PERMISSAO,AREA 1,116,271110 - PONTE RASA/METRO PATRIARCA,62,452,24,6,0,0,544,75,286,31,936
689,2020-01-08,CONCESSAO,AREA 3,310,536221 - Grajaú - Pça da Sé,18,331,9,1,0,0,359,102,45,0,506


In [ ]:
a= all_data.groupby(['Data'])['Passageiros Pagtes Em Dinheiro'].sum()
b = all_data.groupby(['Data'])['Passageiros Comum e VT'].sum()
c = all_data.groupby(['Data'])['Passageiros Pgts Bu Comum M'].sum()
d = all_data.groupby(['Data'])['Passageiros Pagtes Estudante'].sum()
e = all_data.groupby(['Data'])['Passageiros Pgts Bu Est Mensal'].sum()
f = all_data.groupby(['Data'])['Passageiros Pgts Bu Vt Mensal'].sum()
g = all_data.groupby(['Data'])['Passageiros Pagantes'].sum()
h = all_data.groupby(['Data'])['Passageiros Int Ônibus->Ônibus'].sum()
i = all_data.groupby(['Data'])['Passageiros Com Gratuidade'].sum()
j = all_data.groupby(['Data'])['Passageiros Com Gratuidade Est'].sum()
t = all_data.groupby(['Data'])['Tot Passageiros Transportados'].sum()
i1 = pd.concat([t,a,b,c,d,e,f,g,h,i,j],axis = 1)
i1.to_excel('tabela_total_de_passageiros_transportados_janeiro_2021.xls', sheet_name='Sheet1')

In [ ]:
i1

,Tot Passageiros Transportados,Passageiros Pagtes Em Dinheiro,Passageiros Comum e VT,Passageiros Pgts Bu Comum M,Passageiros Pagtes Estudante,Passageiros Pgts Bu Est Mensal,Passageiros Pgts Bu Vt Mensal,Passageiros Pagantes,Passageiros Int Ônibus->Ônibus,Passageiros Com Gratuidade,Passageiros Com Gratuidade Est
Data,,,,,,,,,,,
2021-01-01,877454,84494,397561,17876,1514,13,4,501462,210339,157636,8017
2021-01-02,1815828,125376,830799,41245,2431,26,9,999886,456253,347793,11896
2021-01-03,1386627,105135,621265,30039,2110,25,1,758575,341408,275949,10695
2021-01-04,4432888,274548,2136161,82392,8629,65,22,2501817,1184035,720161,26875
2021-01-05,4158465,239427,2010232,78778,7865,57,24,2336383,1118931,678718,24433
2021-01-06,4922912,267385,2408380,97367,9073,51,31,2782287,1347168,793427,30
2021-01-07,4949754,270547,2423838,97274,8515,64,25,2800263,1357240,792247,4
2021-01-08,4968541,284173,2404853,98279,8045,46,18,2795414,1354410,818716,1
2021-01-09,2941804,198394,1406558,67222,4539,23,5,1676741,773528,491535,0


#**Ambiente de execução**

In [ ]:
arquivos = glob.glob('/content/drive/My Drive/Prefeitura/Resultados/2019/*.xls')
df = pd.DataFrame(arquivos)
arquivos

['/content/drive/My Drive/Prefeitura/Resultados/2019/2-tabela_total_de_passageiros_transportados_fevereiro_2019.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/2019/3-tabela_total_de_passageiros_transportados_marco_2019.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/2019/4-tabela_total_de_passageiros_transportados_abril_2019.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/2019/5-tabela_total_de_passageiros_transportados_maio_2019.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/2019/7-tabela_total_de_passageiros_transportados_julho_2019.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/2019/6-tabela_total_de_passageiros_transportados_junho_2019.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/2019/8-tabela_total_de_passageiros_transportados_agosto_2019.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/2019/9-tabela_total_de_passageiros_transportados_setembro_2019.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/2019/10-tabela_total_de_passageir

In [ ]:
x = len(arquivos)
print("Existem", x, "arquivos nessa lista.")
lin = df.iloc[0,0]
print("URL número 1 desta lista é:", lin)

Existem 11 arquivos nessa lista.
URL número 1 desta lista é: /content/drive/My Drive/Prefeitura/Resultados/2019/2-tabela_total_de_passageiros_transportados_fevereiro_2019.xls


In [ ]:
all_data = pd.DataFrame() 
for f in arquivos:
  df1 = pd.read_excel(f,skiprows = range(0, 2))
  all_data = all_data.append(df1)
all_data.sample(2)

,2019-02-02 00:00:00,5332887,377544,1053024,128838,21017,78,68,2843756,1560098,867083,61950,2019-03-02 00:00:00,4855425,353693,856958,119239,50095,123,64,2647666,1119665,814940,273154,2019-04-02 00:00:00,8705790,435131,1514971,201903,223783,182,130,4494255,2001711,1380101,829723,2019-05-02 00:00:00,9052702,487886,1567207,...,4431841,1949548,1293161,790836,2019-10-02 00:00:00,9302013,478685,3823631,236126,265564,133,75,4804214,2104508,1507481,885810,2019-11-02 00:00:00,4233002,283339,1668849,114771,70795,59,36,2137849,981502,801967,311684,2019-12-02 00:00:00,8757269,493058,3648204,213431,212752,163,41,4567649,2007225,1453885,728510
6,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,...,4705091.0,2090066.0,1504695.0,843845.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,...,2752665.0,1219350.0,867844.0,342564.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
a= all_data.groupby(['Data'])['Passageiros Pagtes Em Dinheiro'].sum()
b = all_data.groupby(['Data'])['Passageiros Comum e VT'].sum()
c = all_data.groupby(['Data'])['Passageiros Pgts Bu Comum M'].sum()
d = all_data.groupby(['Data'])['Passageiros Pagtes Estudante'].sum()
e = all_data.groupby(['Data'])['Passageiros Pgts Bu Est Mensal'].sum()
f = all_data.groupby(['Data'])['Passageiros Pgts Bu Vt Mensal'].sum()
g = all_data.groupby(['Data'])['Passageiros Pagantes'].sum()
h = all_data.groupby(['Data'])['Passageiros Int Ônibus->Ônibus'].sum()
i = all_data.groupby(['Data'])['Passageiros Com Gratuidade'].sum()
j = all_data.groupby(['Data'])['Passageiros Com Gratuidade Est'].sum()
t = all_data.groupby(['Data'])['Tot Passageiros Transportados'].sum()
i1 = pd.concat([t,a,b,c,d,e,f,g,h,i,j],axis = 1)
i1 = i1.sort_values(by=['Data'])
i1.to_excel('12-tabela_total_de_passageiros_transportados_dezembro_2019.xls', sheet_name='Sheet1')

In [ ]:
i1

,Tot Passageiros Transportados,Passageiros Pagtes Em Dinheiro,Passageiros Comum e VT,Passageiros Pgts Bu Comum M,Passageiros Pagtes Estudante,Passageiros Pgts Bu Est Mensal,Passageiros Pgts Bu Vt Mensal,Passageiros Pagantes,Passageiros Int Ônibus->Ônibus,Passageiros Com Gratuidade,Passageiros Com Gratuidade Est
Data,,,,,,,,,,,
2019-12-01,3225535,253809,1267627,90282,48635,71,16,1660440,721538,624695,218862
2019-12-02,8757269,493058,3648204,213431,212752,163,41,4567649,2007225,1453885,728510
2019-12-03,9049424,458465,3783485,222750,231452,125,39,4696316,2088299,1513686,751123
2019-12-04,9013339,438207,3785384,223845,229717,121,51,4677325,2085504,1513583,736927
2019-12-05,8907609,446150,3785056,217157,223816,146,42,4672367,2063688,1465531,706023
2019-12-06,9021275,514354,3828799,215738,218094,145,36,4777166,2095753,1527605,620751
2019-12-07,6048570,443938,2528014,155417,116238,120,40,3243767,1366414,1026372,412017
2019-12-08,3459212,282968,1360336,90778,60802,60,23,1794967,766458,662810,234977
2019-12-09,8618495,494999,3683221,202234,199131,141,44,4579770,2010692,1450693,577340


teste

In [ ]:
url = '/content/drive/My Drive/Prefeitura/Resultados/fato_total_passageiros_transportados_sptrans.csv'
df = pd.read_csv(url)
df.head()

,Data,Tot_Passageiros_Transportados,Passageiros_Pagtes_Em_Dinheiro,Passageiros_Comum_e_VT,Passageiros_Pgts_Bu_Comum_M,Passageiros_Pagtes_Estudante,Passageiros Pgts Bu Est Mensal,Passageiros_Pgts_Bu_Vt_Mensal,Passageiros_Pagantes,Passageiros_Int_Onibus_Onibus,Passageiros_Com_Gratuidade,Passageiros Com Gratuidade Est
0,2019-02-01 00:00:00,8565761,504393,1773956,187179,37560,124,116,4569310,2592601,1341236,62614
1,2019-02-02 00:00:00,5332887,377544,1053024,128838,21017,78,68,2843756,1560098,867083,61950
2,2019-02-03 00:00:00,2872336,222311,477045,70681,10738,52,46,1462656,824588,547817,37275
3,2019-02-04 00:00:00,7424481,461956,1469170,158432,54315,115,88,3956575,2191541,1120494,155871
4,2019-02-05 00:00:00,2796390,158655,578766,55897,22701,33,28,1428611,822060,479052,66667


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3 non-null      object
dtypes: object(1)
memory usage: 152.0+ bytes


#**Ambiente de junção por partes.**

In [ ]:
arquivos = glob.glob('/content/drive/My Drive/Prefeitura/Resultados/Anos/*.xls')
df = pd.DataFrame(arquivos)
arquivos

['/content/drive/My Drive/Prefeitura/Resultados/Anos/3-tabela_total_de_passageiros_transportados_2021.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/Anos/2-tabela_total_de_passageiros_transportados_2020.xls',
 '/content/drive/My Drive/Prefeitura/Resultados/Anos/1-tabela_total_de_passageiros_transportados_2019 (1).xls']

In [ ]:
x = len(arquivos)
print("Existem", x, "arquivos nessa lista.")
lin = df.iloc[0,0]
print("URL número 1 desta lista é:", lin)

Existem 3 arquivos nessa lista.
URL número 1 desta lista é: /content/drive/My Drive/Prefeitura/Resultados/Anos/3-tabela_total_de_passageiros_transportados_2021.xls


In [ ]:
all_data = pd.DataFrame() 
for f in arquivos:
  df1 = pd.read_excel(f)
  all_data = all_data.append(df1)
all_data = all_data.sort_values(by=['Data'])
all_data.set_index('Data',inplace=True)
all_data.sample(2)

,Tot Passageiros Transportados,Passageiros Pagtes Em Dinheiro,Passageiros Comum e VT,Passageiros Pgts Bu Comum M,Passageiros Pagtes Estudante,Passageiros Pgts Bu Est Mensal,Passageiros Pgts Bu Vt Mensal,Passageiros Pagantes,Passageiros Int Ônibus->Ônibus,Passageiros Com Gratuidade,Passageiros Com Gratuidade Est
Data,,,,,,,,,,,
2021-02-22,5430244,322964,2740540,115361,16526,99,26,3195516,1585854,601271,47603
2019-02-03,2872336,222311,477045,70681,10738,52,46,1462656,824588,547817,37275


In [ ]:
all_data.rename(columns={'Passageiros Pagtes Em Dinheiro':'passageiros_pagtes_em_dinheiro'}, inplace = True)
all_data.rename(columns={'Passageiros Comum e VT':'passageiros_comum_e_vt'}, inplace = True)
all_data.rename(columns={'Passageiros Pgts Bu Comum M':'passageiros_pgts_bu_comum_m'}, inplace = True)
all_data.rename(columns={'Passageiros Pagtes Estudante':'passageiros_pagtes_estudante'}, inplace = True)
all_data.rename(columns={'Passageiros Pgts Bu Est Mensall':'passageiros_pgts_bu_est_mensal'}, inplace = True)
all_data.rename(columns={'Passageiros Pgts Bu Vt Mensal':'passageiros_pgts_bu_vt_mensal'}, inplace = True)
all_data.rename(columns={'Passageiros Pagantes':'passageiros_pagantes'}, inplace = True)
all_data.rename(columns={'Passageiros Int Ônibus->Ônibus':'passageiros_int_onibus_onibus'}, inplace = True)
all_data.rename(columns={'Passageiros Com Gratuidade':'passageiros_com_gratuidade'}, inplace = True)
all_data.rename(columns={'Passageiros Com Gratuidade Est':'passageiros_com_gratuidade_est'}, inplace = True)
all_data.rename(columns={'Tot Passageiros Transportados':'total_passageiros_transportados'}   , inplace = True) 

In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 849 entries, 2019-02-01 to 2021-05-31
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype
---  ------                           --------------  -----
 0   total_passageiros_transportados  849 non-null    int64
 1   passageiros_pagtes_em_dinheiro   849 non-null    int64
 2   passageiros_comum_e_vt           849 non-null    int64
 3   passageiros_pgts_bu_comum_m      849 non-null    int64
 4   passageiros_pagtes_estudante     849 non-null    int64
 5   Passageiros Pgts Bu Est Mensal   849 non-null    int64
 6   passageiros_pgts_bu_vt_mensal    849 non-null    int64
 7   passageiros_pagantes             849 non-null    int64
 8   passageiros_int_onibus_onibus    849 non-null    int64
 9   passageiros_com_gratuidade       849 non-null    int64
 10  passageiros_com_gratuidade_est   849 non-null    int64
dtypes: int64(11)
memory usage: 79.6 KB


In [ ]:
all_data.to_excel('0-tabela_total_de_passageiros_transportados_2019_2020_2021.xls', sheet_name='Sheet1')
df.to_csv(r'fato_total_passageiros_transportados_sptrans.csv', index = False)
all_data

,total_passageiros_transportados,passageiros_pagtes_em_dinheiro,passageiros_comum_e_vt,passageiros_pgts_bu_comum_m,passageiros_pagtes_estudante,Passageiros Pgts Bu Est Mensal,passageiros_pgts_bu_vt_mensal,passageiros_pagantes,passageiros_int_onibus_onibus,passageiros_com_gratuidade,passageiros_com_gratuidade_est
Data,,,,,,,,,,,
2019-02-01,8565761,504393,1773956,187179,37560,124,116,4569310,2592601,1341236,62614
2019-02-02,5332887,377544,1053024,128838,21017,78,68,2843756,1560098,867083,61950
2019-02-03,2872336,222311,477045,70681,10738,52,46,1462656,824588,547817,37275
2019-02-04,7424481,461956,1469170,158432,54315,115,88,3956575,2191541,1120494,155871
2019-02-05,2796390,158655,578766,55897,22701,33,28,1428611,822060,479052,66667
...,...,...,...,...,...,...,...,...,...,...,...
2021-05-27,4427952,217975,2271333,94440,20154,52,6,2603960,1306542,470036,47414
2021-05-28,4993222,261976,2544320,104594,21387,48,7,2932332,1456678,552234,51978
2021-05-29,2961450,192481,1486602,70362,9421,41,3,1758910,827888,342216,32436


Teste 3 - não funciounou (arquivo)

In [ ]:
all_data = pd.DataFrame()
total = pd.DataFrame()
x = 0
while x < 10:
    lin2 = df.iloc[x,0]
    bs = pd.read_excel(lin2,skiprows = range(0, 2))
    dados1 = bs.groupby(['Data'])['Tot Passageiros Transportados'].sum()
    dados2 = bs.groupby(['Data'])['Passageiros Comum e VT'].sum()
    i1 = pd.concat([dados1,dados2],axis = 1)
    all_data = all_data.append(i1,ignore_index=True)
    x = x + 1

teste 4 - quase funcionou

In [ ]:
all_data = pd.DataFrame(columns=('Data','a','b','c','d','e','f','g','h','i','j','t'))
all_data.index = all_data.Data
all_data = all_data.drop("Data", 1)
l = 0
all_data

,a,b,c,d,e,f,g,h,i,j,t
Data,,,,,,,,,,,


In [ ]:
while l < x:
    lin2 = str(df.iloc[l,0])
    bs = pd.read_excel(lin2,skiprows = range(0, 2))
    a= bs.groupby(['Data'])['Passageiros Pagtes Em Dinheiro'].sum()
    b = bs.groupby(['Data'])['Passageiros Comum e VT'].sum()
    c = bs.groupby(['Data'])['Passageiros Pgts Bu Comum M'].sum()
    d = bs.groupby(['Data'])['Passageiros Pagtes Estudante'].sum()
    e = bs.groupby(['Data'])['Passageiros Pgts Bu Est Mensal'].sum()
    f = bs.groupby(['Data'])['Passageiros Pgts Bu Vt Mensal'].sum()
    g = bs.groupby(['Data'])['Passageiros Pagantes'].sum()
    h = bs.groupby(['Data'])['Passageiros Int Ônibus->Ônibus'].sum()
    i = bs.groupby(['Data'])['Passageiros Com Gratuidade'].sum()
    j = bs.groupby(['Data'])['Passageiros Com Gratuidade Est'].sum()
    t = bs.groupby(['Data'])['Tot Passageiros Transportados'].sum()
    i1 = pd.concat([a,b,c,d,e,f,g,h,i,j,t],axis = 1)
    i1 = i1.fillna(0)
    i1.rename(columns={'Passageiros Pagtes Em Dinheiro':'a'}, inplace = True)
    i1.rename(columns={'Passageiros Comum e VT':'b'}, inplace = True)
    i1.rename(columns={'Passageiros Pgts Bu Comum M':'c'}, inplace = True)
    i1.rename(columns={'Passageiros Pagtes Estudante':'d'}, inplace = True)
    i1.rename(columns={'Passageiros Pgts Bu Est Mensal':'e'}, inplace = True)
    i1.rename(columns={'Passageiros Pgts Bu Vt Mensal':'f'}, inplace = True)
    i1.rename(columns={'Passageiros Pagantes':'g'}, inplace = True)
    i1.rename(columns={'Passageiros Int Ônibus->Ônibus':'h'}, inplace = True)
    i1.rename(columns={'Passageiros Com Gratuidade':'i'}, inplace = True)
    i1.rename(columns={'Passageiros Com Gratuidade Est':'j'}, inplace = True)
    i1.rename(columns={'Tot Passageiros Transportados':'t'}, inplace = True) 
    i1 = i1.infer_objects()
    all_data = all_data.append(i1,ignore_index=True)
    l += 1

#Ambiente de verificação das variaveis

In [ ]:
lin2

'/content/drive/My Drive/Prefeitura/compilado_base/2021/Janeiro/31JAN2021.xls'

In [ ]:
l

31

In [ ]:
all_data

,0
0,/content/drive/My Drive/Prefeitura/Resultados/...
1,/content/drive/My Drive/Prefeitura/Resultados/...
2,/content/drive/My Drive/Prefeitura/Resultados/...
3,/content/drive/My Drive/Prefeitura/Resultados/...
4,/content/drive/My Drive/Prefeitura/Resultados/...
5,/content/drive/My Drive/Prefeitura/Resultados/...
6,/content/drive/My Drive/Prefeitura/Resultados/...
7,/content/drive/My Drive/Prefeitura/Resultados/...
8,/content/drive/My Drive/Prefeitura/Resultados/...
9,/content/drive/My Drive/Prefeitura/Resultados/...


In [ ]:
i1

,a,b,c,d,e,f,g,h,i,j,t
Data,,,,,,,,,,,
2021-01-31,98557,628059,33040,1330,21,0,761007,354854,280519,65,1396445


arquivo

In [ ]:
all_data = pd.DataFrame()
for f in arquivos:
    df = pd.read_excel(f,
                  skiprows = range(0, 2)
                     )
    all_data = all_data.append(df,ignore_index=True)

Ambiente de confirmação de acesso ao google driver

In [ ]:
url2 = "/content/drive/My Drive/Prefeitura/compilado_base/2021/19MAI2021(1).xls"
dados = pd.read_excel(url2,
                  skiprows = range(0, 2) # ignorei as primeiras 2 linhas
                     )
dados.sample(2)

,Data,Tipo,Area,Empresa,Linha,Passageiros Pagtes Em Dinheiro,Passageiros Comum e VT,Passageiros Pgts Bu Comum M,Passageiros Pagtes Estudante,Passageiros Pgts Bu Est Mensal,Passageiros Pgts Bu Vt Mensal,Passageiros Pagantes,Passageiros Int Ônibus->Ônibus,Passageiros Com Gratuidade,Passageiros Com Gratuidade Est,Tot Passageiros Transportados
971,2021-05-19,PERMISSAO,AREA 1,115,372F10 - FAC SAO JUDAS TADEU/METR BRESS,0,1,0,0,0,0,1,0,0,0,1
1146,2021-05-19,PERMISSAO,AREA 1,119,611610 - JD PRAINHA/TERM GRAJAU,29,1165,31,3,0,0,1228,248,136,19,1631


#**Aqui a começa o desenvolvimento para a automatização da leitura**

Para contornar a questão de ler muitos arquivos, vou inciar as variaveis a partir da data atual, subitraindo até chegar na primeira data!

In [ ]:
data_atual = date.today()
data_atual_ano = data_atual.year
data_atual_mes = data_atual.month
data_atual_dia = data_atual.day
print(data_atual_ano)
print(data_atual_mes)
print(data_atual_dia)

In [ ]:
a = data_atual_ano
m = data_atual_mes
d = data_atual_dia

In [ ]:
while (a > 2020):
  while (m > 0):
      url0 = "https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/"
      if m == 12:
        mes = "DEZ"
      elif m == 11:
        mes = "NOV"
      elif m == 10:
        mes = "OUT"
      elif m == 9:
        mes = "SET"
      elif m == 8:
        mes = "AGO"
      elif m == 7:
        mes = "JUL"
      elif m == 6:
        mes = "JUN"
      elif m == 5:
        mes = "MAI"
      elif m == 4:
        mes = "ABR"
      elif m == 3:
        mes = "MAR"
      elif m == 2:
        mes = "FEV"
      elif m = 1:
        mes = "JAN"
      else m = 0:
        a = a - 1
        m = "DEZ"

      data = d+mes+a 
      d(1) = pd.read_excel(url0 + data + ".xls",
                  skiprows = range(0, 2) # ignorei as primeiras 2 linhas
                     )
      
  else:
    Print("Junção dos arquivos de 2021 concluida.")

In [ ]:
for file in glob.glob('https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/22MAI2021.xls*.XML'):

In [ ]:
now = datetime.now()

print("Tupla de Datetime: " + str(now))

td = timedelta(hours=2)
print("Time delta: " + str(td) + "\n")

print("Hora Atual " + str(now))
print("+2 horas " + str(now + td))
print("-2 horas " + str(now - td))

Tupla de Datetime: 2021-05-26 11:50:50.133130
Time delta: 2:00:00

Hora Atual 2021-05-26 11:50:50.133130
+2 horas 2021-05-26 13:50:50.133130
-2 horas 2021-05-26 09:50:50.133130
